# Running Federated Computations in the Cloud

Let's explore how easy it will be to modify Part A to support predictions in the cloud. We'll point out the differences below.

The first steps are to set up the cloud instances. There are few prerequisites that need to be setup prior to launching the cloud instances. This tutorial uses Google Cloud so you'll need an account set up there and need to have the gcloud command-line tool installed.

See [here](../private-prediction/CLOUD.md) for more details about installing the gcloud command-line tool and launching the instances.

NOTE: If unable to use Google Cloud, you can set the environment variables `INSTANCE_NAMES` and use the script `launch_local_servers.sh`

```
export INSTANCE_NAMES="server0 server1 server2 model-owner data-owner-0 data-owner-1 data-owner-2"
./launch_local_servers.sh $INSTANCE_NAMES
```

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

config = tfe.RemoteConfig.load("/tmp/config.json")
config.connect_to_cluster()

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.Pond())

from players import BaseModelOwner, BaseDataOwner
from func_lib import default_model_fn, secure_mean, evaluate_classifier
from util import split_dataset
from data_lib import federate_dataset
from download import download_mnist

NUM_DATA_OWNERS = 3
BATCH_SIZE = 256
DATA_ITEMS = 60000
EPOCHS = 5
ROUNDS = EPOCHS * DATA_ITEMS // NUM_DATA_OWNERS // BATCH_SIZE
LEARNING_RATE = 0.01

MODEL_OWNER = "model-owner"
DATA_OWNERS = ["data-owner-0", "data-owner-1", "data-owner-2"]

In [2]:
class ModelOwner(BaseModelOwner):
  @classmethod
  def model_fn(cls, data_owner):
    return default_model_fn(data_owner)

  @classmethod
  def aggregator_fn(cls, model_gradients, model):
    return secure_mean(model_gradients)

  @classmethod
  def evaluator_fn(cls, model_owner):
    return evaluate_classifier(model_owner)

class DataOwner(BaseDataOwner):
  pass

In [3]:
simulation_tfrecords = federate_dataset(
    "fashion_mnist:3.0.0", 
    DATA_OWNERS, 
    MODEL_OWNER,
    validation_split=.2,
    gcp_send=True,
)

dataowner_tfrecords = {do: simulation_tfrecords[do] for do in DATA_OWNERS}
validation_tfrecords = simulation_tfrecords.get(MODEL_OWNER, None)

CompletedProcess(args="gcloud compute ssh model-owner --command='mkdir -p /tmp/data'", returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='gcloud compute scp /tmp/data/fashion_mnist:3.0.0:train:model-owner.tfrecord model-owner:/tmp/data/fashion_mnist:3.0.0:train:model-owner.tfrecord', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args="gcloud compute ssh data-owner-0 --command='mkdir -p /tmp/data'", returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='gcloud compute scp /tmp/data/fashion_mnist:3.0.0:train:data-owner-0.tfrecord data-owner-0:/tmp/data/fashion_mnist:3.0.0:train:data-owner-0.tfrecord', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args="gcloud compute ssh data-owner-1 --command='mkdir -p /tmp/data'", returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='gcloud compute scp /tmp/data/fashion_mnist:3.0.0:train:data-owner-1.tfrecord data-owner-1:/tmp/data/fashion_mnist:3.0.0:train:data-owner-1.tfrecord', returncode=0, stdout=b'', stder

In [4]:
model = tf.keras.Sequential((
    tf.keras.layers.Dense(512, input_shape=[None, 28 * 28],
                          activation='relu'),
    tf.keras.layers.Dense(10),
))

model.build()

loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(LEARNING_RATE)

In [5]:
model_owner = ModelOwner(MODEL_OWNER,
                         validation_tfrecords,
                         model, loss,
                         optimizer=opt)

data_owners = [DataOwner(data_owner,
                         dataowner_tfrecords[data_owner],
                         model, loss,
                         optimizer=opt)
              for data_owner in DATA_OWNERS]

In [ ]:
tf.device(data_owners)

In [23]:
data_owners[0].device._device_name

'/job:tfe/replica:0/task:4/cpu:0'

In [6]:
dataowner_tfrecords

{'data-owner-0': ['/tmp/data/fashion_mnist:3.0.0:train:data-owner-0.tfrecord'],
 'data-owner-1': ['/tmp/data/fashion_mnist:3.0.0:train:data-owner-1.tfrecord'],
 'data-owner-2': ['/tmp/data/fashion_mnist:3.0.0:train:data-owner-2.tfrecord']}

In [10]:
model_owner.fit(data_owners, rounds=ROUNDS, evaluate_every=10)

print("\nDone training!!")

> /Users/jasonmancuso/tf-world/federated-learning/players.py(124)_update_one_round()
-> try:  # If the DataOwner implements a model_fn, use it
(Pdb) l
119  	
120  	    # One round is an aggregation over all DataOwners
121  	    for owner in data_owners:
122  	      import pdb; pdb.set_trace()
123  	
124  ->	      try:  # If the DataOwner implements a model_fn, use it
125  	        grads = owner.call_model_fn(owner, player_name=owner.player_name,
126  	                                    **kwargs)
127  	        player_gradients.append(grads)
128  	      except NotImplementedError:  # Otherwise, fall back to the ModelOwner's
129  	        grads = self.call_model_fn(owner, player_name=owner.player_name,
(Pdb) n
> /Users/jasonmancuso/tf-world/federated-learning/players.py(125)_update_one_round()
-> grads = owner.call_model_fn(owner, player_name=owner.player_name,
(Pdb) n
> /Users/jasonmancuso/tf-world/federated-learning/players.py(126)_update_one_round()
-> **kwargs)
(Pdb) n
NotImplemented

(Pdb) actual_player_name
'data-owner-0'
(Pdb) compute_func
<function BaseModelOwner.call_model_fn at 0x12f211b00>
(Pdb) compute_func_args
(<__main__.ModelOwner object at 0x12ffbe850>, <__main__.DataOwner object at 0x12ffbe350>)
(Pdb) compute_func_args[1]
(Pdb) compute_func_ars[1].player_name
*** NameError: name 'compute_func_ars' is not defined
(Pdb) compute_func_args[0].player_name
'model-owner'
(Pdb) compute_func_args[1].player_name
'data-owner-0'
(Pdb) kwargs
{}
(Pdb) l
125  	    return compute_func_wrapper
126  	
127  	  if actual_compute_func is None:
128  	    # User has not yet passed a compute_func, so we'll expect them to
129  	    # pass it outside of this function's scope (e.g. as a decorator).
130  	    return decorator
131  	
132  	  # User has already passed a compute_func, so return the decorated version.
133  	  return decorator(actual_compute_func)
134  	
135  	
(Pdb) compute_func_wrapper
*** NameError: name 'compute_func_wrapper' is not defined
(Pdb) l
136  	def set_p

(Pdb) n
> /Users/jasonmancuso/tf-world/venv/lib/python3.7/site-packages/tf_encrypted/protocol/pond/pond.py(591)define_local_computation()
-> with tf.device(player.device_name):
(Pdb) l
586  	      raise TypeError(("Don't know how to process input argument "
587  	                       "of type {}").format(type(x)))
588  	
589  	    with tf.name_scope(name_scope if name_scope else "local-computation"):
590  	
591  ->	      with tf.device(player.device_name):
592  	        if arguments is None:
593  	          inputs = []
594  	        else:
595  	          if not isinstance(arguments, (list, tuple)):
596  	            arguments = [arguments]
(Pdb) player.device_name
'/job:tfe/replica:0/task:4/cpu:0'
(Pdb) n
> /Users/jasonmancuso/tf-world/venv/lib/python3.7/site-packages/tf_encrypted/protocol/pond/pond.py(592)define_local_computation()
-> if arguments is None:
(Pdb) l
587  	                       "of type {}").format(type(x)))
588  	
589  	    with tf.name_scope(name_scope if name_scope

BdbQuit: 